## Advent of code 2022 day 1-10
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import sortedcontainers
#import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines, nostrip=False):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end,
    by default also strips all lines (if nostrip is set only strips empty lines)'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line_str=line.strip()
        if nostrip==False or len(line_str)<1:
            line=line_str
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2022 day 6
# mv ~/Downloads/input* data_src/2022-day-6-input.txt
# big input file looks like: single long line

sample2='''
mjqjpqmgbljsphdztnvjfqwrcgsmlb
bvwbjplbgvbhsrlpgdmjqwftvncz
nppdvjthqldpwncqszvftbrmjlhg
nznrnfrfntjfmvfwmzdfjlvtqnbhcprsg
zcfzfwzzqfrljwzlrfnpqdbhtmscgvjw
'''

def find_marker_pos(line, n):
    for pos in range(0, len(line)-n):
        subs=line[pos:pos+n]
        if len(set(subs))==n:
            return pos+n
    return -1

sample1=open('data_src/2022-day-6-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
print('part 1')
for line in lines:
    print(line[:10], find_marker_pos(line, 4))

print('part 2')
for line in lines:
    print(line[:10], find_marker_pos(line, 14))

# part 1: 1850
# part 2: 2823

In [ ]:
# 2022 day 5
# mv ~/Downloads/input* data_src/2022-day-5-input.txt
# big input file looks like: crates side by side, (at most 9), moving instructions
# idea for part 1: parse head of lines like a board (per column, from the bottom up),
# then execute the moves,
# for part 2: execute the moves with the 'batched' variation

sample2='''
    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2
'''

def parse_cols(lines):
    '''split lines into cols (already parsed crate stacks), and tail'''
    groups=get_line_groups(lines, nostrip=True)
    assert len(groups)==2
    sts=groups[0]
    tail=groups[1]
    assert len(sts)>0 and len(tail)>0
    cols={} # maps col id (num) to list of crate chars, bottom-most first
    for col in range(0, 10):
        x=col*4+1
        y=len(sts)-1
        if x>=len(sts[y]):
            continue
        assert sts[y][x]==str(col+1)
        for y in range(len(sts)-2, -1, -1):
            if x<len(sts[y]) and 'A'<=sts[y][x]<='Z':
                cl=cols.setdefault(col+1, [])
                cl.append(sts[y][x])
    return cols, tail

def parse_moves(tail, cols, batched=False):
    '''perform moves in tail, changing cols, if batched move all crates in same order,
    return resulting message'''
    for line in tail:
        m=re.match(r'move\s*(\d+)\s*from\s*(\d+)\s*to\s*(\d+)', line)
        if m:
            n, src, dst=int(m.group(1)), int(m.group(2)), int(m.group(3))
            assert n>0
            if batched==False:
                for i in range(n):
                    assert len(cols[src])>0
                    c=cols[src].pop(-1)
                    cols[dst].append(c)
            else:
                assert len(cols[src])>=n
                si=len(cols[src])-n
                for i in range(n):
                    c=cols[src].pop(si)
                    cols[dst].append(c)
    msg1=''
    for k in sorted(cols.keys()):
        msg1+=cols[k][-1]
    return msg1

sample1=open('data_src/2022-day-5-input.txt').read()
lines=[s for s in sample1.splitlines() ]
cols, tail=parse_cols(lines)
msg1=parse_moves(tail, cols, batched=False)
print('part 1:', msg1)
cols, tail=parse_cols(lines)
msg2=parse_moves(tail, cols, batched=True)
print('part 2:', msg2)

# part 1: QNHWJVJZW
# part 2: BPCZJLFJW

In [ ]:
# 2022 day 4
# mv ~/Downloads/input* data_src/2022-day-4-input.txt

sample2='''
2-4,6-8
2-3,4-5
5-7,7-9
2-8,3-7
6-6,4-6
2-6,4-8
'''

sample1=open('data_src/2022-day-4-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ re.split(r'[,\-]', s) for s in lines ]
data=[ [ int(s) for s in row ] for row in data ]
count=0
for row in data:
    a, b, c, d=row
    assert a<=b
    assert c<=d
    if (c<=a<=b<=d) or (a<=c<=d<=b):
        count+=1
print('part 1:', count)
count=0
for row in data:
    a, b, c, d=row
    if not (b<c or d<a):
        count+=1
print('part 2:', count)

# part 1 644
# part 2 926

In [ ]:
# 2022 day 3
# mv ~/Downloads/input* data_src/2022-day-3-input.txt
# big input file looks like: list of 300 character strings

sample2='''
vJrwpWtwJgWrhcsFMMfFFhFp
jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
PmmdzqPrVvPwwTWBwg
wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
ttgJtRGJQctTZtZT
CrZsJsPPZsGzwwsLwLmpwMDw
'''

sample1=open('data_src/2022-day-3-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
part1score=0
for s in lines:
    n=len(s)//2
    s1=s[:n]
    s2=s[n:]
    common=set(s1).intersection(set(s2))
    c=list(common)[0]
    score=ord(c)-ord('A')+27 if 'A'<=c<='Z' else ord(c)-ord('a')+1
    #print(s1, s2, common, score)
    part1score+=score
print('part 1', part1score)
part2score=0
for i in range(0, len(lines), 3):
    s1=lines[i]
    s2=lines[i+1]
    s3=lines[i+2]
    common=set(s1).intersection(set(s2)).intersection(set(s3))
    c=list(common)[0]
    score=ord(c)-ord('A')+27 if 'A'<=c<='Z' else ord(c)-ord('a')+1
    part2score+=score
print('part 2', part2score)

# part 1 8515
# part 2 2434

In [ ]:
# 2022 day 2
# mv ~/Downloads/input* data_src/2022-day-2-input.txt
# big input file looks like: long list of pairs of chars

sample2='''
A Y
B X
C Z
'''

sample1=open('data_src/2022-day-2-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ s.split() for s in lines ]
score=0
for tup in data:
    opp, own=tup
    ownnum={'X': 0, 'Y': 1, 'Z': 2}[own]
    score+=ownnum+1
    oppnum={'A': 0, 'B': 1, 'C': 2}[opp]
    if oppnum==ownnum:
        score+=3
    elif ownnum==(oppnum+1)%3:
        score+=6
print('part 1', score)
score=0
for tup in data:
    opp, own=tup
    oppnum={'A': 0, 'B': 1, 'C': 2}[opp]
    ownnum={'X': 0, 'Y': 1, 'Z': 2}[own]
    if ownnum==0:
        ownplay=(oppnum-1)%3
    elif ownnum==1:
        ownplay=oppnum
    else:
        ownplay=(oppnum+1)%3
    score+=ownplay+1
    score+=ownnum*3
print('part 2', score)

# part 1 13052
# part 2 13693

In [ ]:
# 2022 day 1
# mv ~/Downloads/input* data_src/2022-day-1-input.txt
# big input file looks like: long list of groups of lines / numbers

sample2='''
1000
2000
3000

4000

5000
6000

7000
8000
9000

10000
'''

sample1=open('data_src/2022-day-1-input.txt').read()
groups=get_line_groups(sample1.splitlines())
groups=[ sum([int(s) for s in g]) for g in groups]
print(f'part 1: {max(groups)}')
sum3=sum(sorted(groups)[-3:])
print(f'part 2: {sum3}')

#part 1: 72511
#part 2: 212117

In [ ]:
# TEMPLATE
# 2022 day 1
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2022-day-1-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2022-day-1-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
groups=get_line_groups(sample2.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed